In [1]:
# Weak scaling test 4
# 4 nodes
# 36 GB of data
from pyspark.sql import SparkSession
from functools import reduce
import json
import timeit
import pyspark.sql.functions as f 

spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.225:7077") \
        .appName("Weak_scaling_4")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.cores.max",4)\
        .getOrCreate()

In [2]:
%%time
start_time_0 = timeit.default_timer()

df = spark_session.read.json('hdfs://192.168.2.225:9000/reddit')

elapsed_0 = timeit.default_timer() - start_time_0

print("Time elapsed: ", elapsed_0, "s")

Time elapsed:  435.36421260004863 s
CPU times: user 118 ms, sys: 26.7 ms, total: 144 ms
Wall time: 7min 15s


In [3]:
df.count()

64757312

In [4]:
initial_start_time = timeit.default_timer()

if 'author_cakeday' in df.columns:
        current_df = df.drop('author_cakeday')
        
df = df.drop('author_flair_css_class','author_flair_text','can_gild','distinguished','edited','id','is_submitter','link_id','parent_id','permalink','retrieved_on','stickied','subreddit_id')

df = df.filter((df.body != '[deleted]'))


elapsed = timeit.default_timer() - initial_start_time



print("Time elapsed: ", elapsed, "s")

Time elapsed:  0.27475284098181874 s


In [5]:
%%time
start_time_1 = timeit.default_timer()

#df.select("author").show()
df.groupBy('subreddit').count().sort("count", ascending = False).show(10)

elapsed_1 = timeit.default_timer() - start_time_1

print("Time elapsed: ", elapsed_1,"s")

+-------------------+-------+
|          subreddit|  count|
+-------------------+-------+
|          AskReddit|7724254|
|              funny|3149078|
|               pics|2500644|
|             gaming|2028916|
|            atheism|1884228|
|           politics|1646043|
|                WTF|1564693|
|fffffffuuuuuuuuuuuu|1299961|
|              trees|1231457|
|               IAmA|1173054|
+-------------------+-------+
only showing top 10 rows

Time elapsed:  262.26044142700266 s
CPU times: user 75.3 ms, sys: 17.5 ms, total: 92.8 ms
Wall time: 4min 22s


In [6]:
%%time
start_time_2 = timeit.default_timer()

df = df.withColumn('wordCount', f.size(f.split(f.col('body'), ' ')))
#df.show()

df.groupBy('wordCount').count().sort("count", ascending = False).show(10)   

elapsed_2 = timeit.default_timer() - start_time_2


print("Time elapsed: ", elapsed_2,"s")

+---------+-------+
|wordCount|  count|
+---------+-------+
|        5|2353726|
|        6|2344218|
|        4|2270790|
|        7|2256362|
|        8|2144356|
|        1|2082495|
|        3|2074165|
|        9|2017343|
|       10|1892965|
|        2|1854274|
+---------+-------+
only showing top 10 rows

Time elapsed:  366.4533951568883 s
CPU times: user 96 ms, sys: 37.7 ms, total: 134 ms
Wall time: 6min 6s


In [7]:
%%time
start_time_3 = timeit.default_timer()

df_stats = df.select(f.mean(f.col('wordCount')).alias('mean')).collect()  #

mean = df_stats[0]['mean']

print("Average wordcount in comment: ", mean, " words.")


elapsed_3 = timeit.default_timer() - start_time_3
print("Time elapsed: ", elapsed_3,"s")

Average wordcount in comment:  32.177884605623206  words.
Time elapsed:  357.3905587020563 s
CPU times: user 111 ms, sys: 40 ms, total: 151 ms
Wall time: 5min 57s


In [8]:
total_elapsed = timeit.default_timer() - initial_start_time

print("Total time elapsed: ", total_elapsed, "s")

Total time elapsed:  986.4500576358987 s


In [9]:
spark_session.stop()